# Annual change in the water content

In [1]:
import numpy as np
import h5py
import os, sys
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.signal import savgol_filter
import pandas as pd
sys.path.remove(sys.path[0])

sys.path.append('/Users/ajc/Core/codes/simulator/debug/ats/tools/utils/')
import get_transect_data

sys.path.append('/Users/ajc/Core/PreProcessing/PyScript')
import readdata

In [2]:
path = "/Users/ajc/Core/simulations/barrow_icewedges/hydrologic_connection/ProjectionsO/"

In [3]:
#Dec 15, 2020: ProjectionsO
Simulations = [['CESM_C2%s'%i for i in range(3)], ['CESM_C1%s'%i for i in range(4)],['CESM_C3%s'%i for i in range(3)] ]# modified rain model, dessicated zone =0.1, longwaves
Sim_name = ['S1','S2','S3']

In [4]:
def getData(i,var_):
    Data = []
    Time = []
    for s in Simulations[i]:
        infile = os.path.join(path,s)
        
        Tim1, Dat1 = get_transect_data.get2DMatrix(infile,varnames=[var_])
        a1 = len(Dat1[0,:,0,0])%73
        a2 = len(Dat1[0,:,0,0]) - a1
        
        Tim = Tim1[:a2]
        Dat = Dat1[:,:a2,:,:]
        Time.append(Tim)
        Data.append(Dat)     
    d = dict()
    key_v = Sim_name[i] + '_' + var_
    key_t = Sim_name[i] + '_' + 'time'
    d[key_v] = Data
    d[key_t] = Time
    
    return d

In [5]:
#[nvar-1,i,j,zlen-depth_s:zlen-depth_e]
#d1 = getData(0)

In [6]:
from multiprocessing import Pool
from functools import partial
num_processors = 3
pool=Pool(processes = num_processors)

def call_data(var):
    global DataS1, DataS2, DataS3
    prod_x=partial(getData, var_=var)
    DataS1, DataS2, DataS3 = pool.map(prod_x,[i for i in range(len(Simulations))])
    Data = [DataS1, DataS2,DataS3]
    return Data

3 709 45 85
3 1515 45 85
3 2182 45 85
3 4774 45 85
3 3499 45 85
3 4700 45 85
3 1375 45 85
3 803 45 85
3 1387 45 85
3 219 45 85


In [7]:
def getData_All(var):
    DataSimOrig = dict()
    DataWC = call_data(var)
    for i in range(len(Sim_name)): #number of simulations
        data_wc = []
        L = len(Simulations[i])
        Time = []
        Wcontent  = []
        for k in range(L): #k is for the number of directories for a simulation
            key = list(DataWC[i].keys())[0] #[0] ='water_content', [1] = 'time'
            
            key_time = list(DataWC[i].keys())[1] 
            #print (i, key)
            data_time = np.array(DataWC[i][key_time][k])
            data_wc = np.array(DataWC[i][key][k])
            
            #print (data_wc.shape,data_temp.shape,data_time.shape)
            nvar, cycles, xnum, znum = data_wc.shape

            count = 1
            count_summer = 0
            sum1 = 0
            for cyc in range(cycles):
                wc = np.sum(data_wc[nvar-1,cyc,:,:])
                Wcontent.append(wc)
                Time.append(data_time[cyc])
                
        DataSimOrig[Sim_name[i] + '_water_content'] = np.array(Wcontent)
        DataSimOrig[Sim_name[i] + '_time'] = np.array(Time)
    return DataSimOrig

In [9]:
DataSim = getData_All(var='water_content')

In [10]:
def get_moving_average(d, m=5):
    d_avg = []
    len1 = len(d)
    for i in range(len1):
        if i < m:
            s1 = np.mean(d[:i+1])
        else:
            s1= np.mean(d[i-m:i])

        d_avg.append(s1)
    return d_avg

In [11]:
def getSurfaceData(i):
    Data = []
    var ='surface-water_content'
    for s in Simulations[i]:
        infile = os.path.join(path,s,'visdump_surface_data.h5')
        dx=h5py.File(infile,'r')
        Dat11 = np.array(readdata.GetSurfVarFromVis_All(infile,var)[var])
        Dat11 = Dat11.mean(axis=1)
        a1 = len(Dat11)%73
        a2 = len(Dat11) - a1

        Dat = Dat11[:a2]
        Data.append(Dat)
        
    d = dict()
    d[var] = np.concatenate(Data)
    
    return d

In [12]:
from multiprocessing import Pool
from functools import partial
num_processors = 3
pool=Pool(processes = num_processors)

def call_data_surface():
    global DataSurf1, DataSurf2, DataSurf3
    prod_x=partial(getSurfaceData)
    DataSurf1, DataSurf2, DataSurf3 = pool.map(prod_x,[i for i in range(len(Simulations))])
    Data = [DataSurf1, DataSurf2, DataSurf3]
    return Data

In [13]:
call_data_surface()

[{'surface-water_content': array([1320.46366377, 1320.45353671, 1320.45353671, ...,    0.        ,
            0.        ,    0.        ])},
 {'surface-water_content': array([1320.46366377, 1320.45353671, 1320.45353671, ...,    0.        ,
            0.        ,    0.        ])},
 {'surface-water_content': array([1320.46366377, 1320.46366377, 1320.46366377, ..., 1254.68674286,
         1254.6819328 , 1254.67896053])}]

In [71]:
%matplotlib qt
outfile='/Users/ajc/Documents/MyPapers/TroughHydrodynamics/figures/Dec22_uniform_om/'
def Plot_WaterContent():
    fig, axs = plt.subplots(1,1, figsize=(5,3.5), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    
    plt.tight_layout()
    #axs = axs.ravel()
    xpos = np.array([0, 1,2,3])

    Labels = ['no-flow', 'free-flow', 'hybrid-flow']
    Labels = ['zero-head','specified-head','dynamic-head']
    Colors = ['k','b','r','r']
    T = ['a','b','c','d']
    Keys = list(DataSim.keys())
    DataSurf = [DataSurf1, DataSurf2, DataSurf3]
    Data_temp = []
    for i in range(3):
        ss_d = DataSim[Keys[2*i]]
        sf_d = DataSurf[i]['surface-water_content']
        
        d = np.array([x+y for x,y in zip(ss_d,sf_d)])
        dx = np.reshape(d, (-1,73))
        
        storage_chanage = []
        for d1 in dx:
            storage_chanage.append(d1[0]/dx[0][0])
        
        d_avg5yr = get_moving_average(storage_chanage,m=5)
        
        x = np.linspace(2006,2100,95,dtype='i')
        axs.plot(x,d_avg5yr,color=Colors[i],label=Labels[i])
        
        axs.set_xlim((2006, 2100))
        axs.set_xlabel('Year',fontsize=12, fontweight='normal')
        
        axs.set_ylabel('Normalized water content [-]',fontsize=12, fontweight='normal')
        axs.legend(loc='upper left',labelspacing=.1)
        Data_temp.append(d_avg5yr)
        print('---------')
    Data_temp1 = np.reshape(Data_temp,(3,-1))
    plt.savefig(outfile+'normalized-moisture_domain-5yr_avg.pdf', bbox_inches='tight', dpi=500)
    #np.savetxt(outfile+'normalized-moisture_ALT-5yr_avg.dat', Data_temp1)
    plt.show()



In [72]:
Plot_WaterContent()

---------
---------
---------


# STOP HERE

In [20]:
6935/73.

95.0